In [ ]:
# https://mp.weixin.qq.com/s?__biz=MzA4OTAwMjY2Nw==&mid=2650190180&idx=1&sn=15cb1e3a7b0e6f7b22bf5114884a4178&chksm=882396a0bf541fb69125c34a0d211c8c513da0aa6abb84a76ef2b10c974276310f78b485e9a0&scene=21#wechat_redirect

In [218]:
import numpy as np
import pandas as pd
from csv import reader
import time
import cpca
import jieba
from gensim import corpora, models, similarities

In [219]:
df = pd.read_excel("中小企业板.xlsx", dtype={'公司代码': 'str'})
df.columns
# Index(['公司代码', '公司简称', '公司全称', '英文名称', '注册地址', 'A股代码', 'A股简称', 'A股上市日期',
# 'A股总股本', 'A股流通股本', '地 区', '省 份', '城 市', '所属行业', '公司网址'],
# dtype='object')

/Applications/anaconda/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Index(['公司代码', '公司简称', '公司全称', '英文名称', '注册地址', 'A股代码', 'A股简称', 'A股上市日期',
       'A股总股本', 'A股流通股本', 'B股代码', 'B股 简 称', 'B股上市日期', 'B股总股本', 'B股流通股本',
       '地      区', '省    份', '城     市', '所属行业', '公司网址'],
      dtype='object')

In [220]:
def read_data():
    #导入数据,并对数据做一些处理
    df = pd.read_excel("中小企业板.xlsx", dtype={'公司代码': 'str'})
    addr_df = df[["公司代码", "公司简称", "注册地址"]]
    addr_df["注册地址"]= addr_df["注册地址"].apply(lambda x: str(x).strip())
    
    return addr_df

In [221]:

def get_dataset(addr_df):
    '''
    #对地址做一个标准化处理,其中导入cpca的包进行处理
    '''
    start = time.process_time()
    
    location_str = []
    for i in addr_df['注册地址']:
        location_str.append(i.strip())
        
    addr_cp = cpca.transform(location_str)
    
    #给结果表拼接唯一识别代码
    e_data = addr_df[["公司代码", "公司简称"]]
    addr_cpca = pd.concat([e_data, addr_cp], axis=1)
    
    #1.区不为空
    addr_cpca_1 = addr_cpca[(addr_cpca['省']!= '')&(addr_cpca['市']!= '') & (addr_cpca['区']!= '')]
    addr_cpca_1= addr_cpca_1.dropna()
    
    addr_cpca_11= addr_cpca_1[(addr_cpca['地址']!='')]
    addr_cpca_12= addr_cpca_11. dropna(subset=['地址'])
    
    #将前三个字段完全拼接在一起进行分组然后组内进行相似度分析
    addr_cpca_12['省市区'] = addr_cpca_12['省'] + addr_cpca_12['市'] + addr_cpca_12['区']
    
    addr_cpca_12['省市区长度']=addr_cpca_12['省市区'].apply(lambda x: len(x))
    count_1 = addr_cpca_12['省市区'].value_counts().reset_index()
    count_1= count_1.rename(columns={'index':'省市区', '省市区':'省市区'})
    
#     count_delete_1= count_1[count_1['个数']==1]
    dataset_1 = pd.merge(addr_cpca_12, count_1, on = '省市区', how = 'left')
    dataset_1= dataset_1[dataset_1['count']!=1]
    
#     #2.区为空  
#     addr_cpca_2 = addr_cpca[(addr_cpca['省']!= '')&(addr_cpca['市']!= '') & (addr_cpca['区']== '')]
#     addr_cpca_2 = addr_cpca_2.dropna()
    
#     addr_cpca_21= addr_cpca_2[(addr_cpca['地址']!='')]
#     addr_cpca_22= addr_cpca_21. dropna(subset=['地址'])
    
#     #将前三个字段完全拼接在一起进行分组然后组内进行相似度分析
#     addr_cpca_22['省市区'] = addr_cpca_22['省'] + addr_cpca_22['市']
    
#     addr_cpca_22['省市区长度']=addr_cpca_22['省市区'].apply(lambda x: len(x))
#     count_2 = addr_cpca_22['省市区'].value_counts().reset_index()
#     count_2= count_2.rename(columns={'index':'省市区', '省市区':'个数'})
    
#     count_delete_2 = count_2[count_2['个数']==1]
#     dataset_2 = pd.merge(addr_cpca_22, count_delete_2, on = '省市区', how = 'left')
#     dataset_2 = dataset_2[dataset_2['个数']!=1]
    
#     print("Time used:", (time.process_time()()-start), "s")
    
    return dataset_1

In [222]:
def cal_similar(doc_goal, document, ssim = 0.1):
#def cal_similar(doc_goai, document):
    '''
    分词;计算文本相似度
    doc_goal,短文本,目标文档
    document,多个文本,被比较的多个文档
    '''
    all_doc_list=[]
    for doc in document:
        doc= "".join(doc)
        doc_list=[word for word in jieba.cut(doc)]
        all_doc_list.append(doc_list)

    #目标文档
    doc_goal = "".join(doc_goal)
    doc_goal_list = [word for word in jieba.cut(doc_goal)]
    
    #被比较的多个文档
    dictionary = corpora.Dictionary(all_doc_list)  #先用dictionary方法获词袋
    corpus = [dictionary.doc2bow(doc) for doc in all_doc_list]  #使用doc2bow制作预料库
    
    #目标文档
    doc_goal_vec = dictionary.doc2bow(doc_goal_list)
    tfidf = models.TfidfModel(corpus)#使用TF-DF模型对料库建模
    index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features = len(dictionary.keys()))#对每个目标文档,分析测文档的相似度
    
    #开始比较
    sims = index[tfidf[doc_goal_vec]]
    #similary= sorted(enumerate(sims),key=lambda item: -item[1])#根据相似度排序
    
    addr_dict={"被比较地址": document, "相似度": list(sims)}
    similary = pd.DataFrame(addr_dict)
    similary["目标地址"] = doc_goal
    similary_data = similary[["目标地址", "被比较地址", "相似度"]]
    similary_data= similary_data[similary_data["相似度"]>=ssim]
    
    return similary_data

In [223]:
def cycle_first(single_data):
    
    single_value = single_data.loc[:,["公司代码","地址"]].values #提取地址
    
    cycle_data = []
    for key, value in enumerate(single_value):
        if key < len(single_data)-1:
            doc_goal=list(value)[1:]
            document=list(single_data["地址"])[key+1:]
            cycle = cal_similar(doc_goal, document, ssim=0)
            cycle['目标地址代码'] = list(single_data["公司代码"])[key]
            cycle['被比较地址代码'] = list(single_data["公司代码"])[key+1:]
            cycle = cycle[["目标地址代码","目标地址", "被比较地址代码", "被比较地址", "相似度"]]
        #print("循环第",key,"个地址,得到表的行数,",len(cycle),",当前子循环计算进度,",key/len(cycle))
        cycle_data.extend(cycle.values.tolist())
    cycle_data=pd.DataFrame(cycle_data,columns=["目标地址代码","目标地址", "被比较地址代码", "被比较地址", "相似度"])
    cycle_data = cycle_data.drop_duplicates()   
    return cycle_data

In [224]:
def get_collect(dataset):

#     start = time. clock()
    
    #获取单个省市区的文档
    collect_data = []
    ssq=list(set(dataset_1['省市区']))
    for v, word in enumerate(ssq):
        single_data = dataset_1[dataset_1['省市区'] == word]
    #     print("循环第",v,"个省市区地址为:",word,",当前此区地址有:",len(single_data),",当前计算进度为:{:.1f}%" .format(v*100/len(ssq)))
        cycle_data = cycle_first(single_data)
        collect_data.extend(cycle_data.values.tolist())#将每个市区得到的结果放入一张表
    collect_data=pd.DataFrame(collect_data,columns=["目标地址代码","目标地址", "被比较地址代码", "被比较地址", "相似度"])
    collect_data
    
    return collect_data

In [225]:
def run_(par = 0):
    #调用上述函数
    addr_df = read_data()
    dataset_1 = get_dataset(addr_df)
    #dataset. to_csv("../data/addr_data/document_address.csv", index =False)
    #dataset. to_csv( ". /data/addr_data/document_address. csv", index False)
    collect_data_1 = get_collect(dataset_1)
#     collect_data_2 = get_collect(dataset_2)
    collect_data = pd.concat([collect_data_1], axis=0)
    collect_data = collect_data[collect_data["相似度"]>=par].sort_values(by=["相似度"], ascending=[False])
    
    collect_data["相似度"] = collect_data["相似度"].apply(lambda x: ('%.2f' % x))

    return collect_data

In [226]:
collect_data = run_(par = 0.1)


/Applications/anaconda/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/var/folders/q2/74vp1g0x6gz96k_qr9vcsdp80000gn/T/ipykernel_76520/2307475006.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addr_df["注册地址"]= addr_df["注册地址"].apply(lambda x: str(x).strip())
/var/folders/q2/74vp1g0x6gz96k_qr9vcsdp80000gn/T/ipykernel_76520/960814751.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  addr_cpca_11= addr_cpca_1[(addr_cpca['地址']!='')]


In [227]:
collect_data

,目标地址代码,目标地址,被比较地址代码,被比较地址,相似度
1186,002203,店口镇工业区,002590,店口镇工业区,1.00
957,002659,西杉创意园四区2号楼西段一层109,002933,西杉创意园四区4号楼,0.90
1206,002222,软件大道89号福州软件园F区9号楼,002512,软件大道89号福州软件园G区17号楼,0.87
1544,002170,高新技术园学府路63号联合总部大厦30-31楼,002587,粤海街道学府路63号高新区联合总部大厦10楼,0.86
876,002410,西北旺东路10号院东区13号楼,002649,西北旺东路10号院东区7号楼博彦科技大厦,0.80
...,...,...,...,...,...
1591,002191,科技中二路劲嘉科技大厦18-19层,002886,南头关口二路智恒战略性新兴产业园10栋1至2楼,0.10
831,002383,知春路118号知春大厦1501室,002649,西北旺东路10号院东区7号楼博彦科技大厦,0.10
1139,002797,福华一路115号投行大厦20楼,002856,八卦四路科研楼7栋1-6层,0.10
2253,002813,海天一路11号5栋C座8楼、9楼,002970,学苑大道1001号南山智园B1栋21-23楼,0.10


In [229]:
collect_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 540 entries, 1186 to 297
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   目标地址代码   540 non-null    object
 1   目标地址     540 non-null    object
 2   被比较地址代码  540 non-null    object
 3   被比较地址    540 non-null    object
 4   相似度      540 non-null    object
dtypes: object(5)
memory usage: 25.3+ KB


In [230]:
collect_data['相似度']=collect_data['相似度'].astype(float)

In [231]:
df_relation

,:START_ID,:END_ID,:TYPE
1186,002203,002590,1.00
957,002659,002933,0.90
1206,002222,002512,0.87
1544,002170,002587,0.86
876,002410,002649,0.80
1191,002263,002590,0.78
1639,002210,002313,0.77
1811,002313,002359,0.77
75,002312,002773,0.75
2298,002970,002979,0.74


In [232]:
'''
#对地址做一个标准化处理,其中导入cpca的包进行处理
'''
start = time.process_time()

location_str = []
for i in addr_df['注册地址']:
    location_str.append(i.strip())

In [233]:
location_str

['浙江省新昌县羽林街道江北路4号',
 '江苏省扬州市广陵区杭集镇曙光路',
 '浙江省临海市花园工业区',
 '重庆市渝北区人和星光大道69号',
 '安徽省蚌埠市燕南路1308号',
 '浙江省绍兴市柯桥区鉴湖路1809号',
 '河南省新乡市华兰大道甲1号',
 '广东省深圳市南山区深南大道9988号',
 '江苏省无锡市惠山区洛社镇洛藕路288号',
 '浙江省杭州市萧山经济技术开发区',
 '浙江省诸暨市店口工业区',
 '浙江省遂昌县妙高镇凯恩路1008号',
 '湖北省襄阳市高新区追日路8号',
 '安徽省黄山市徽州区徽州东路188号',
 '江苏省江阴市徐霞客镇马镇东街7号',
 '广东省珠海市斗门区井岸镇珠峰大道288号1区17号楼',
 '广东省珠海市高新区南屏科技工业园屏工中路8号',
 '浙江省临安经济开发区',
 '浙江省新昌县羽林街道新昌大道东路800号',
 '浙江省玉环市大麦屿街道兴港东路198号',
 '上海市徐汇区钦州北路1189号',
 '四川省成都市高新区高朋大道21号',
 '江苏省南京市山西路8号金山大厦1-5层',
 '贵州省贵阳经济技术开发区红河路7号',
 '山东省威海临港经济技术开发区苘山镇中韩路2号',
 '广东省广州市黄埔区中新知识城凤凰三路8号2号楼2130室',
 '上海市闵行区金都路4399号',
 '福建省晋江市金井镇南工业区',
 '广东省广州市高新技术开发区科学城香山路19号',
 '广东省揭东经济开发区5号路中段',
 '浙江省玉环县大麦屿经济开发区',
 '云南省丽江市古城区香格里大道760号丽江玉龙旅游大厦',
 '浙江省湖州市天字圩路288号',
 '广东省中山市小榄镇工业大道南华园路1号',
 '江西省南昌市南昌高新技术产业开发区京东大道1699号',
 '贵州省贵阳市高新技术开发区新天园区',
 '北京市海淀区西三环北路100号金玉大厦1103-1105室',
 '贵州省贵阳市都司高架桥路46号黔源大厦',
 '江苏省南京市栖霞区和燕路251号1幢1101室',
 '山东省莱州市农科院城山路西侧',
 '安徽省淮北市经济开发区新区石山路6号',
 '浙江省德清县洛舍镇工业区',
 '江苏省南通市人民东路218号',
 '广东省广州市花都区新雅街镜湖大道8号',
 '河南省洛阳市高新技术

In [235]:
addr_cp = cpca.transform(location_str)

In [236]:
addr_cp

,省,市,区,地址,adcode
0,浙江省,绍兴市,新昌县,羽林街道江北路4号,330624
1,江苏省,扬州市,广陵区,杭集镇曙光路,321002
2,浙江省,台州市,临海市,花园工业区,331082
3,重庆市,市辖区,渝北区,人和星光大道69号,500112
4,安徽省,蚌埠市,None,燕南路1308号,340300
...,...,...,...,...,...
954,湖南省,长沙市,雨花区,迎新路868号德思勤城市广场第A-2地块第7栋16层,430111
955,北京市,市辖区,海淀区,西三环北路25号7层,110108
956,广东省,清远市,None,高新技术产业开发区泰基工业城,441800
957,广东省,深圳市,福田区,沙头街道天安社区泰然十路天安创新科技广场二期东座1601、1605,440304


In [237]:
addr_cpca

,公司代码,公司简称,省,市,区,地址,adcode
0,002001,新 和 成,浙江省,绍兴市,新昌县,羽林街道江北路4号,330624
1,002002,鸿达兴业,江苏省,扬州市,广陵区,杭集镇曙光路,321002
2,002003,伟星股份,浙江省,台州市,临海市,花园工业区,331082
3,002004,华邦健康,重庆市,市辖区,渝北区,人和星光大道69号,500112
4,002005,*ST德豪,安徽省,蚌埠市,None,燕南路1308号,340300
...,...,...,...,...,...,...,...
954,002986,宇新股份,湖南省,长沙市,雨花区,迎新路868号德思勤城市广场第A-2地块第7栋16层,430111
955,002987,京北方,北京市,市辖区,海淀区,西三环北路25号7层,110108
956,002988,豪美新材,广东省,清远市,None,高新技术产业开发区泰基工业城,441800
957,002990,盛视科技,广东省,深圳市,福田区,沙头街道天安社区泰然十路天安创新科技广场二期东座1601、1605,440304


In [238]:
addr_cpca_12

,公司代码,公司简称,省,市,区,地址,adcode,省市区,省市区长度
0,002001,新 和 成,浙江省,绍兴市,新昌县,羽林街道江北路4号,330624,浙江省绍兴市新昌县,9
1,002002,鸿达兴业,江苏省,扬州市,广陵区,杭集镇曙光路,321002,江苏省扬州市广陵区,9
2,002003,伟星股份,浙江省,台州市,临海市,花园工业区,331082,浙江省台州市临海市,9
3,002004,华邦健康,重庆市,市辖区,渝北区,人和星光大道69号,500112,重庆市市辖区渝北区,9
5,002006,精功科技,浙江省,绍兴市,柯桥区,鉴湖路1809号,330603,浙江省绍兴市柯桥区,9
...,...,...,...,...,...,...,...,...,...
953,002985,北摩高科,北京市,市辖区,昌平区,科技园区火炬街甲12号B218室,110114,北京市市辖区昌平区,9
954,002986,宇新股份,湖南省,长沙市,雨花区,迎新路868号德思勤城市广场第A-2地块第7栋16层,430111,湖南省长沙市雨花区,9
955,002987,京北方,北京市,市辖区,海淀区,西三环北路25号7层,110108,北京市市辖区海淀区,9
957,002990,盛视科技,广东省,深圳市,福田区,沙头街道天安社区泰然十路天安创新科技广场二期东座1601、1605,440304,广东省深圳市福田区,9


In [ ]:
#给结果表拼接唯一识别代码
e_data = addr_df[["公司代码", "公司简称"]]
addr_cpca = pd.concat([e_data, addr_cp], axis=1)

#1.区不为空
addr_cpca_1 = addr_cpca[(addr_cpca['省']!= '')&(addr_cpca['市']!= '') & (addr_cpca['区']!= '')]
addr_cpca_1= addr_cpca_1.dropna()

addr_cpca_11= addr_cpca_1[(addr_cpca['地址']!='')]
addr_cpca_12= addr_cpca_11. dropna(subset=['地址'])

#将前三个字段完全拼接在一起进行分组然后组内进行相似度分析
addr_cpca_12['省市区'] = addr_cpca_12['省'] + addr_cpca_12['市'] + addr_cpca_12['区']

addr_cpca_12['省市区长度']=addr_cpca_12['省市区'].apply(lambda x: len(x))
count_1 = addr_cpca_12['省市区'].value_counts().reset_index()
count_1= count_1.rename(columns={'index':'省市区', '省市区':'省市区'})

#     count_delete_1= count_1[count_1['个数']==1]
dataset_1 = pd.merge(addr_cpca_12, count_1, on = '省市区', how = 'left')
dataset_1= dataset_1[dataset_1['count']!=1]

#     #2.区为空  
#     addr_cpca_2 = addr_cpca[(addr_cpca['省']!= '')&(addr_cpca['市']!= '') & (addr_cpca['区']== '')]
#     addr_cpca_2 = addr_cpca_2.dropna()

#     addr_cpca_21= addr_cpca_2[(addr_cpca['地址']!='')]
#     addr_cpca_22= addr_cpca_21. dropna(subset=['地址'])

#     #将前三个字段完全拼接在一起进行分组然后组内进行相似度分析
#     addr_cpca_22['省市区'] = addr_cpca_22['省'] + addr_cpca_22['市']

#     addr_cpca_22['省市区长度']=addr_cpca_22['省市区'].apply(lambda x: len(x))
#     count_2 = addr_cpca_22['省市区'].value_counts().reset_index()
#     count_2= count_2.rename(columns={'index':'省市区', '省市区':'个数'})

#     count_delete_2 = count_2[count_2['个数']==1]
#     dataset_2 = pd.merge(addr_cpca_22, count_delete_2, on = '省市区', how = 'left')
#     dataset_2 = dataset_2[dataset_2['个数']!=1]

#     print("Time used:", (time.process_time()()-start), "s")

return dataset_1